In [1]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style(style="dark")

Using TensorFlow backend.


### Environment

**STATE**

* Cart Position
* Cart Velocity
* Pole Angle
* Pole Velocity at tip

**ACTIONS**
* 0 - LEFT
* 1 - RIGHT

**REWARD**

* 1 for every survived step

### Build Agent

In [2]:
def build_Deep_Q_Agent():
    model = Sequential()
    model.add(Dense(32, input_dim=state_size, activation='relu'))
    model.add(Dense(8, input_dim=state_size, activation='relu'))
    model.add(Dense(action_size, activation='linear')) # LEFT OR RIGHT
    model.compile(loss='mse', optimizer=Adam(lr=learning_rate))
    return model

### Perform action from given state

In [3]:
def perform_action(agent, state, epsilon):
    action = -1
    
    if np.random.rand() <= epsilon:
        action = random.randrange(2)
    else:
        # max(Q(s,a)) - State-action function
        action = agent.predict(state)
        action = np.argmax(action[0]) # [0.74, 0.2] LEFT / RIGHT
        
    return action

### Remember transition

In [4]:
def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done)) # Replay memory, D

### Train agent to maximize future reward

In [5]:
def replay(batch_size, epsilon, done):
    
    # Get a minibatch of previous training memories
    minibatch = random.sample(memory, batch_size)
    
    # For each memory
    for state, action, reward, next_state, done in minibatch:
        
        if not done:
            # Predict from next_state the (maximal) FUTURE DISCOUNTED REWARD
            next_state_pred_reward = (reward + gamma * np.amax(agent.predict(next_state)[0]))
        else:
            next_state_pred_reward = reward
        
        
        # Predict from current state the FUTURE DISCOUNTED REWARD depeding on action which was taken during training
        current_state_pred_reward = agent.predict(state) # Predicted rewards [22.2112, 20.22112]
        current_state_pred_reward[0][action] = next_state_pred_reward # Set PREDICTED FUTURE REWARD for the action which was taken
        
        # next_state_pred_reward = [36] (MAX, LEFT)
        # current_state_pred_reward = [25, 23]
        # current_state_pred_reward[action] = [36, 23]
        
        # This will make the agent learn the FUTURE DISCOUNTED REWARD based on current state
        agent.fit(state, current_state_pred_reward, epochs=1, verbose=0)
    
    # Decrease exploration successively
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay
    
    return epsilon

### Load and Save

In [6]:
def load(model, name):
    return model.load_weights(name)

def save(model, name):
    model.save_weights(name)

### Params

In [7]:
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
memory = deque(maxlen=10000) # D-memory (transitions)
gamma = 0.95 # discount rate
epsilon = 0.5 # exploration rate
epsilon_min = 0.01 # min exploration rate
epsilon_decay = 0.995 # decrease exploration successively
learning_rate = 0.0005
episodes = 1000
batch_size = 25
max_score = 0

/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


## Run

In [8]:
#sns.lineplot(range(len(results)), results)

In [9]:
done = False
load_weights = False
results = []

agent = build_Deep_Q_Agent()

if load_weights:
    load(agent, 'weights/cartpole-dqn-50.h5')
    epsilon = 0.01

for episode in range(episodes):

    state = env.reset()
    state = np.reshape(state, [1, state_size])

    for frame in range(episodes):
        env.render()
        #env.reset()
        
        # ---------------------------------------------------------
        # Select action - combine prediction and randomness
        # ---------------------------------------------------------
        
        action = perform_action(agent, state, epsilon)
        next_state, reward, done, info = env.step(action)
        if done: reward = -1
            
        # ---------------------------------------------------------
        # Move to next state and save transition in memory
        # ---------------------------------------------------------
            
        next_state = np.reshape(next_state, [1, state_size])
        remember(state, action, reward, next_state, done) 
        state = next_state
        
        if done:
            
            results.append(frame)
            
            if episode % 25 == 0: print("\nAverage score: ", sum(results[-25:]) / 25.0)
            if frame > max_score: max_score = frame
            
            print("\nEpisode: {}/{}, Score: {}, Epsilon: {:.3}, Top score: {}".format(episode, episodes, frame, float(epsilon), max_score))
            
            break
            
        # ---------------------------------------------------------
        # Sample random minibatches from memory and train model to 
        # maximize future reward from current state
        # ---------------------------------------------------------

        if len(memory) > batch_size:
            epsilon = replay(batch_size, epsilon, done)

    #if episode % 25 == 0:
    #    print("\nSave weights: " + str(episode))
    #    save(agent, "weights/cartpole-dqn-" + str(episode) + ".h5")


Average score:  0.96

Episode: 0/1000, Score: 24, Epsilon: 0.5, Top score: 24

Episode: 1/1000, Score: 29, Epsilon: 0.432, Top score: 29

Episode: 2/1000, Score: 57, Epsilon: 0.325, Top score: 57

Episode: 3/1000, Score: 18, Epsilon: 0.297, Top score: 57

Episode: 4/1000, Score: 20, Epsilon: 0.269, Top score: 57

Episode: 5/1000, Score: 25, Epsilon: 0.237, Top score: 57

Episode: 6/1000, Score: 30, Epsilon: 0.204, Top score: 57

Episode: 7/1000, Score: 25, Epsilon: 0.18, Top score: 57

Episode: 8/1000, Score: 41, Epsilon: 0.146, Top score: 57

Episode: 9/1000, Score: 71, Epsilon: 0.103, Top score: 71

Episode: 10/1000, Score: 88, Epsilon: 0.066, Top score: 88

Episode: 11/1000, Score: 77, Epsilon: 0.0449, Top score: 88

Episode: 12/1000, Score: 126, Epsilon: 0.0239, Top score: 126

Episode: 13/1000, Score: 160, Epsilon: 0.0107, Top score: 160

Episode: 14/1000, Score: 202, Epsilon: 0.00997, Top score: 202

Episode: 15/1000, Score: 130, Epsilon: 0.00997, Top score: 202

Episode: 16/100

KeyboardInterrupt: 